In [1]:
#utf-8 DES block cipher

import nbimporter,os
from AES_singleblock_encrypt import aes_encrypt
from AES_singleblock_decrypt import aes_decrypt

class AES():
    #initialization
    def __init__(self,mode):
        self.mode = mode
        self.enc_list = list()
        self.dec_list = list()
         
    #XOR operation 32 hex digits (4x4 byte)
    def XOR(self,hex1,hex2):
        xor_result = str()
        bit1 = bin(int(hex1,16))[2:].zfill(128)
        bit2 = bin(int(hex2,16))[2:].zfill(128)
        for index in range(len(bit1)):
            r = (int(bit1[index])+int(bit2[index])) % 2
            xor_result = xor_result + str(r)
        xor_result_to_hex = hex(int(xor_result,2))[2:].zfill(32)
        return xor_result_to_hex

    def encrypt(self,plain_txt,key_32hex,iv = None):
        '''block size = 16 bytes'''
        utf8_list = list()
        padding_str = str()
        grou_list = list()
        
        '''transform plaintext into hex string using utf-8'''
        for i in plain_txt:
            if ord(i) < 128:
            #ascii/codepoint 127 :delet
                utf8_list.append(hex(ord(i)).replace('0x','').zfill(2))
            else:
                #utf8
                utf8_cha1 = str(i.encode('utf8')).strip('b')
                utf8_cha2 = utf8_cha1.replace('\\x','')
                utf8_cha3 = utf8_cha2.strip('\'')
                utf8_list.append(utf8_cha3)
        utf8_str = ''.join(utf8_list)
        
        '''PKCS7 padding to reach multiple of 4x4x2 hex digits'''
        padding_str = utf8_str
        if len(utf8_str) % 32 != 0:
            remainder = len(utf8_str) % 32
            padding_num = int((32-remainder)/2)
            for i in range(padding_num):
                padding_str += hex(padding_num)[2:].zfill(2)
    
        '''32 hex digits a group'''
        quotient = int(len(padding_str)/32)
        for j in range(quotient):
            item = padding_str[32*j]
            for k in range(31):
                item += padding_str[32*j + (k+1)]
            grou_list.append(item)

        '''mode operation'''
        if self.mode == 'ecb':
            for plain_32hex in grou_list:
                enc = aes_encrypt(plain_32hex,key_32hex)
                self.enc_list.append(enc.generate())
            return ''.join(self.enc_list)
        
        elif self.mode == 'cbc':
            #first block XOR with initializtion vector
            xor_ob = iv  
            for plain_32hex in grou_list:
                enc = aes_encrypt(self.XOR(xor_ob,plain_32hex),key_32hex)
                xor_ob = enc.generate()
                self.enc_list.append(xor_ob)
            return ''.join(self.enc_list)

    def decrypt(self,cipher_txt,key_32hex,iv = None):
        '''block size = 16 bytes'''
        grou_list = list()
        code_point_list = list()
        bin_str = str()
        
        '''32 hex digits a group'''
        quotient = int(len(cipher_txt)/32)
        for j in range(quotient):
            item = cipher_txt[32*j]
            for k in range(31):
                item += cipher_txt[32*j + (k+1)]
            grou_list.append(item)
 
        '''mode operation'''
        if self.mode == 'ecb':
            for cipher_32hex in grou_list:
                dec = aes_decrypt(cipher_32hex,key_32hex)
                self.dec_list.append(dec.generate())
        
        elif self.mode == 'cbc':
            #first block XOR with initializtion vector
            xor_ob = iv
            for cipher_32hex in grou_list: 
                dec = aes_decrypt(cipher_32hex,key_32hex)
    
                #XOR operation
                xor_result = self.XOR(xor_ob,dec.generate())
                self.dec_list.append(xor_result)
                xor_ob = cipher_32hex
        
        '''2 hex digits a group in list'''
        dec_str = ''.join(self.dec_list)
        quotient = int(len(dec_str)/2)
        for i in range(quotient):
            item = dec_str[2*i] + dec_str[2*i+1]
            code_point_list.append(int(item,16))
        
        '''eliminate padding digits'''
        padding = True
        index,num_pad = len(code_point_list)-1,1
    
        while padding:
            if code_point_list[index] == code_point_list[index-1]:
                index = index-1
                num_pad = num_pad+1
            else:
                if num_pad == code_point_list[index]:
                    for i in range(num_pad):
                        code_point_list.pop()        
                padding = False
        
        '''utf8 decoding'''
        import bitstring as bit
        for j in code_point_list:
            #append 1 byte at a time
            bin_str = bin_str + bin(j)[2:].zfill(8)
    
        '''turn string type to bytes data type'''
        decode_to_bin = bit.BitArray('0b'+ bin_str)  
        decode_result = decode_to_bin.tobytes().decode('utf8')
        return decode_result

#-----------testing-----------------
plaintext = 'Test測試Ã cryptography'
key_32hex = '5468617473206D79204B756E67204675'
iv = '3468617473206D79204B756E67204675'

a = AES(mode='ecb')
a_ecb_cipher = a.encrypt(plaintext,key_32hex)
a_ecb_decipher = a.decrypt(a_ecb_cipher,key_32hex)

b = AES(mode='cbc')
b_cbc_cipher = b.encrypt(plaintext,key_32hex,iv)
b_cbc_decipher = b.decrypt(b_cbc_cipher,key_32hex,iv)

print({a_ecb_cipher:a_ecb_decipher})
print({b_cbc_cipher:b_cbc_decipher})

Importing Jupyter notebook from AES_singleblock_encrypt.ipynb
Importing Jupyter notebook from AES_key_schedule.ipynb
Importing Jupyter notebook from AES_singleblock_decrypt.ipynb
{'8869C95000A53114AF6CEF6F2A8E7D0D890FB3DACE8E1C290BCA49CC09F76E3C': 'Test測試Ã cryptography'}
{'36E1EEA4547C5159CE674CB2F2B6B7EE3E408613F64A549665B346B84054C64E': 'Test測試Ã cryptography'}
